# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [ ]:
import glob
import numpy as np
import scipy as sp
import scipy.io
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import butter, filtfilt, medfilt


# Function to get the raw  data files from dataset



def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

data_fls, ref_fls=LoadTroikaDataset()

# Functions to extract the PPG and Accelerometer data from a data file

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]





def LoadTroikaReferenceFile(ref_fl):
    """
    Loads and extracts signals from a troika reference file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ref_hr = LoadTroikaDataFile(ref_fls[0])

    Args:
        ref_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for reference heart rates.
    """
    data = sp.io.loadmat(ref_fl)['BPM0']
    return data


# Function to calculate average reference pulse rate


def PPG_data(data_fl):
    """
    Loads combined PPG and Accelerometer data and 
    return PPG signals
    """
    data=LoadTroikaDataFile(data_fl)
    return data[0,:]



def Accelerometer_data(data_fl):
    """
    Loads combined PPG and Accelerometer data and 
    return Accelerometer signals
    """
    data=LoadTroikaDataFile(data_fl)
    return data[1:,:]



# ### PPG Data Pre-processing


def BandpassFilter(sig,passband,fs):
    """
    Takes signal, bandpass filter range and sample frequency as the inputs 
    and returns filtered signal as the output
    """
    b,a=sp.signal.butter(5,passband, btype='bandpass',fs=fs)
    return sp.signal.filtfilt(b, a, sig)


# Functions to calculate FFT magnitudes and frequencies for PPG and Accelerometer signal


def FFT_PPG(data_fl,ref_fl,fs,w_overlap,passband):
    
    """
    Computes FFT magnitude and frequencies for PPG signals

    Args:
        data_fl: filepath to a data .mat file
        ref_fl: filepath to a reference .mat file
        fs: sampling rate or frequency (Hz)
        passband: band pass frequency range
    Returns:
        fft_sig_ppg (tuple with list of FFT magnitudes array in the first element and frequencies in second element)
    """
    ppg_i=PPG_data(data_fl)
    ref_heart_rate=LoadTroikaReferenceFile(ref_fl)
    filt_ppg_i=BandpassFilter(ppg_i,passband,fs)
    
    c=len(ref_heart_rate)
    n=int(np.floor((len(filt_ppg_i)+w_overlap)/len(ref_heart_rate)))
    fftmag_ppg=[]
    fft_freq_ppg=np.fft.rfftfreq(n,1/fs)
    i=0
    while (i+n<len(filt_ppg_i)+w_overlap):
        fftmag_ppg.append(np.abs(np.fft.rfft(filt_ppg_i[i:i+n])))
        i=i+n
    #fftmag_ppg=np.abs(np.fft.rfft(filt_ppg_i))
    #fft_freq_ppg=np.fft.rfftfreq(len(filt_ppg_i),1/fs)
    #NFFT = 2**int(np.log(len(filt_ppg_i)/len(ref_heart_rate))/np.log(2))
    #NFFT=int(len(filt_ppg_i)/len(ref_heart_rate))
    #print (NFFT)
    #fft_sig_ppg=plt.specgram(filt_sig_i,NFFT=NFFT,Fs=fs,noverlap=0)
    #plt.clf()
    
    fft_sig_ppg=(fftmag_ppg,fft_freq_ppg)
    return fft_sig_ppg


def FFT_Accelerometer(data_fl,ref_fl,fs,w_overlap,passband):
    
    """
    Computes FFT magnitude and frequencies for accelerometer signals

    Args:
        data_fl: filepath to a data .mat file
        ref_fl: filepath to a reference .mat file
        fs: sampling rate or frequency (Hz)
        passband: band pass frequency range
    Returns:
        fft_sig_acc (tuple) with FFT magnitude of 3 accelerations (accx,accy,accz) and frequencies (numpy arrays) in the second element
        """
    
    acc=Accelerometer_data(data_fl)
    ref_heart_rate=LoadTroikaReferenceFile(ref_fl)
    filt_acc_x=BandpassFilter(acc[0],passband,fs)
    filt_acc_y=BandpassFilter(acc[1],passband,fs)
    filt_acc_z=BandpassFilter(acc[2],passband,fs)
    
    c=len(ref_heart_rate)
    n=int(np.floor((len(filt_acc_x)+w_overlap)/len(ref_heart_rate)))
    acc_mag_total=[]
    fftmag_acc_x=[]
    fftmag_acc_y=[]
    fftmag_acc_z=[]
    fft_freq_ppg=np.fft.rfftfreq(n,1/fs)
    i=0
    while (i+n<len(filt_acc_x)+w_overlap):
        fftmag_acc_x.append(np.abs(np.fft.rfft(filt_acc_x[i:i+n])))
        fftmag_acc_y.append(np.abs(np.fft.rfft(filt_acc_y[i:i+n])))
        fftmag_acc_z.append(np.abs(np.fft.rfft(filt_acc_z[i:i+n])))
        #acc_mag_total.append(np.sqrt(np.square(fftmag_acc_x)+np.square(fftmag_acc_y)+np.square(fftmag_acc_z)))
        i=i+n

    fft_freq_acc=np.fft.rfftfreq(n,1/fs)
    fft_sig_acc=(fftmag_acc_x,fftmag_acc_y,fftmag_acc_z,fft_freq_acc)
    return fft_sig_acc


# Function to find two dominant frequencies


def dominant_freq(sig_mag,sig_freq):
    
    """
    Computes Top two best dominant frequency

    Args:
        sig_mag: FFT magnitude of a signal (numpy array)
        sig_freq: FFT frequencies of a signal (numpy array)
    Returns:
        Top 3 best dominant frequencies
    """
    
    x=np.argsort(sig_mag)[::-1]
    max_freq_1,max_freq_2=sig_freq[x[0:2]]
    return max_freq_1,max_freq_2


# In[77]:


def find_best_frequency(data_fl,ref_fl,fs,w_overlap,passband):
    
    """
    Computes best dominant frequency by comparing accelerometer and PPG signals

    Args:
        data_fls: filepath to data .mat files
        fs: sampling rate or frequency (Hz)
        passband: band pass frequency range
    Returns:
        best dominant frequency
    """
    
    
    fft_sig_ppg=FFT_PPG(data_fl,ref_fl,fs,w_overlap,passband)
    fft_sig_acc=FFT_Accelerometer(data_fl,ref_fl,fs,w_overlap,passband)
    window_size=len(fft_sig_ppg[0])
    s=2
    ppg_dom_freqs=np.zeros(shape=(window_size,s))
    acc_dom_freqs_x=np.zeros(shape=(window_size,s))
    acc_dom_freqs_y=np.zeros(shape=(window_size,s))
    acc_dom_freqs_z=np.zeros(shape=(window_size,s))
    best_freq=np.zeros(shape=(window_size,1))
    
    for i in range(window_size):

        acc_dom_freqs_x[i,:]=dominant_freq(fft_sig_acc[0][i],fft_sig_acc[3])
        acc_dom_freqs_y[i,:]=dominant_freq(fft_sig_acc[1][i],fft_sig_acc[3])
        acc_dom_freqs_z[i,:]=dominant_freq(fft_sig_acc[2][i],fft_sig_acc[3])
        ppg_dom_freqs[i,:]=dominant_freq(fft_sig_ppg[0][i],fft_sig_ppg[1])
        
        # select second best dominant frequency from PPG signal if dominant frequencies of accelerometer and PPG match
        if (acc_dom_freqs_x[i,0]==ppg_dom_freqs[i,0])or (acc_dom_freqs_y[i,0]==ppg_dom_freqs[i,0]) or (acc_dom_freqs_z[i,0]==ppg_dom_freqs[i,0]):       
        #if (ppg_dom_freqs[i,0] in acc_dom_freqs_x[i,:])or (ppg_dom_freqs[i,0] in acc_dom_freqs_y[i,:]) or (ppg_dom_freqs[i,0] in acc_dom_freqs_z[i,:]):           
            best_freq[i,:]=ppg_dom_freqs[i,1]
        else:
            best_freq[i,:]=ppg_dom_freqs[i,0]
            
    return best_freq



def confidence_estimation(mag,freq,pulse_rate,window):
    """
    Computes confidence estimates.

    Args:
        mag: FFT magnitude of a signal
        freq: FFT frequencies of a signal
        pulse_rate: esitmated pulse rate
        window= window around pulse rate estimate (in Hz)

    Returns:
        conf_est: Sum of frequency spectrum around pulse rate estimate/total sum of entire spectrum
    """
    pulse_freq=pulse_rate/60
    selected_mag=mag[(freq<pulse_freq+window) & (freq>pulse_freq-window)]
    conf_est=np.sum(selected_mag)/np.sum(mag)
    return conf_est
    


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))



def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
            
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)



def RunPulseRateAlgorithm(data_fl, ref_fl):
    # Load data using LoadTroikaDataFile
    passband=[40/60,240/60]
    w_overlap=0
    fs=125
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ref_heart_rate=LoadTroikaReferenceFile(ref_fl)
    fft_ppg=FFT_PPG(data_fl,ref_fl,fs,w_overlap,passband)
    
    # Compute pulse rate estimates and estimation confidence.
    pulse_rates_0=pd.Series(60*find_best_frequency(data_fl,ref_fl,fs,w_overlap,passband)[:,0])
    pulse_rates=pulse_rates_0.ewm(com=5).mean().values

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    errors=np.abs(pulse_rates-ref_heart_rate[:,0])
    confidence=[]
    for i in range(len(fft_ppg[0])):
        confidence.append(confidence_estimation(fft_ppg[0][i],fft_ppg[1],pulse_rates[i],window=0.6))
    
    return errors, confidence
